In [1]:
import pandas as pd
import polars as pl

import numpy as np

import lectura_archivos as fr


# Analisis de Cantidad de Traqueostomias por Servicio de Salud

- En este apartado se quiere ver la cantidad de traqueostomias que se hicieron por Servicio de Salud.
Este analisis se realiza por anio.

## Carga de Archivos de GRD

In [2]:
dfp = fr.leer_grd()

In [3]:
df = dfp.to_pandas()

## Analisis de Traqueostomia

In [4]:
procedimientos = df.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"]
contiene_nuestros_proc = procedimientos.isin(["31.1", "31.29"]).sum(axis=1).astype(bool)
proc_nacionales = df[contiene_nuestros_proc]

In [5]:
metricas = (
    proc_nacionales.groupby(["ANIO_EGRESO", "SERVICIO_SALUD"])
    .agg(
        n_egresos=("COD_HOSPITAL", "count"),
        peso_medio=("IR_29301_PESO", "mean"),
        estancia_media=("ESTANCIA", "mean"),
    )
    .sort_values(["ANIO_EGRESO", "n_egresos"], ascending=False)
)


In [6]:
metricas.to_excel("output/resumen_traqueostomia_hist.xlsx")


# Analisis de Prestaciones por Diagnostico

- En este apartado se quiere ver la cantidad de prestaciones que se realizan por cada diagnostico
principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|

## Analisis Exploratorio

### Obtener conteo de diagnosticos en el Torax

In [7]:
torax = dfp.filter(
    (pl.col("COD_HOSPITAL") == 112103) & (pl.col("TIPO_ACTIVIDAD") != "HOSPITALIZACIÓN DIURNA")
)
torax.groupby(["DIAGNOSTICO1"]).agg(pl.count()).sort(by="count", descending=True)

DIAGNOSTICO1,count
str,u32
"""I25.1""",852
"""Z51.1""",811
"""U07.1""",706
"""Z51.4""",594
"""Z29.1""",394
"""C34.1""",367
"""I35.0""",353
"""Z45.0""",305
"""J84.1""",244


- De estos resultados, se puede ver que el diagnostico I25.1 es el diagnostico mas frecuente
entre 2019 y 2021. Tuvo 852 egresos.

- A modo de ejemplo, se quiere ver el dataframe completo del diagnostico Z51.4, y analizar
los procedimientos que tiene incluidos.

### Analisis de conteo de procedimientos

### Forma 1

In [8]:
from functools import reduce

i251 = torax.filter(pl.col("DIAGNOSTICO1") == "I25.1").to_pandas()
listas_procedimientos = (
    i251.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"].fillna("").agg("-".join, axis=1).str.split("-")
)


sumado = reduce(lambda x, y: x + y, listas_procedimientos.values)


### Forma 2

In [9]:
cambiada = pd.melt(
    i251, id_vars="DIAGNOSTICO1", value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)]
)
cambiada["value"].value_counts()

99.29    687
99.19    627
89.54    551
90.59    483
93.96    449
        ... 
90.93      1
78.61      1
39.98      1
99.10      1
37.87      1
Name: value, Length: 256, dtype: int64

## Analisis Completo

- Se quiere ver la cantida dde procedimientos hechos por posicion de procedimiento. O sea,
en el "PROCEDIMIENTO1" que procedimientos se ingresaron, y cuales son sus frecuencias ordenadas
de mayor a menor?

- Tambien se quiere ver la frecuencia global. O sea, para el diagnostico X, cuales son los
procedimientos que mas se le realizan, independiente de la posicion del procedimiento.

#### Cambio de nombre de columnas PROCEDIMIENTO

- Esto se realiza, ya que permite ordenar estas columnas al utilizar .value_counts() en la
proxima celda.

In [10]:
import string

procedimientos = [f"PROCEDIMIENTO{i}" for i in range(1, 31)]
letras = list(string.ascii_uppercase) + ["a", "b", "c", "d"]
codificacion = dict(zip(procedimientos, letras))
codificacion_reversa = dict(zip(letras, procedimientos))

torax_pandas = torax.to_pandas()
long_prestaciones = pd.melt(
    torax_pandas,
    id_vars=["ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=procedimientos,
)

long_prestaciones["variable"] = long_prestaciones["variable"].replace(codificacion)


In [11]:
por_procedimiento = (
    long_prestaciones.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "variable"])["value"]
    .value_counts()
    .reset_index(name="n_procedimientos")
)

por_procedimiento["variable"] = por_procedimiento["variable"].replace(codificacion_reversa)
por_procedimiento = por_procedimiento.rename({"value": "codigo_procedimiento"}, axis=1)

### Obtener diccionarios CIE

In [12]:
cie_9 = pd.read_excel("input/CIE-9.xlsx", dtype={"Código": str})

cie_10 = pd.read_excel("input/CIE-10.xlsx")

In [13]:
unido_proc = por_procedimiento.merge(
    cie_9,
    how="inner",
    left_on="codigo_procedimiento",
    right_on="Código",
    suffixes=("", "procedimientos"),
)

unido_diag = unido_proc.merge(
    cie_10, how="left", left_on="DIAGNOSTICO1", right_on="Código", suffixes=("", "_diags")
)

In [106]:
unido_diag.to_csv(
    "output/prestaciones_por_procedimiento.csv", sep=";", encoding="latin-1", index=False
)

### Procedimientos para diagnosticos mas relevantes

In [17]:
DIAGS_MAS_RELEVANTES = [
        "I34.0",
        "I42.0",
        "J95.5",
        "Q67.6",
        "E84.8",
        "I08.1",
        "J93.1",
        "C34.1",
        "I71.0",
        "C38.4",
        "J90",
        "I71.2",
        "C38.1",
        "C34.3",
        "J84.1",
        "C45.0",
        "J43.9",
        "J39.8",
        "C78.0",
        "T82.0",
        "I35.2",
        "J91",
        "I08.0",
        "Q23.1",
        "C34.0",
        "I47.2",
        "Q21.1",
        "J86.0",
        "D14.3",
        "I05.1",
        "I35.1",
        "J84.9",
        "J47",
        "J67.9",
        "C78.2",
        "C33",
        "I49.5",
        "I45.6",
        "M34.8",
        "I35.0",
        "Z45.0",
        "D38.1",
        "J98.0",
        "J84.8",
        "C34.2",
        "J86.9"
      ]

In [19]:
proc_mas_relevantes = unido_diag[unido_diag['Código_diags'].isin(DIAGS_MAS_RELEVANTES)]
proc_mas_relevantes.to_csv(
    "output/proc_mas_relevantes.csv", sep=";", encoding="latin-1", index=False
)

### Procedimientos para diagnosticos mas complejos